In [2]:
!pip install tensorflow
!pip install tensorflow-probability




In [10]:
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

tfd = tfp.distributions

# Load breast cancer dataset
data = datasets.load_breast_cancer()
X, y = data.data, data.target
X = X.astype(np.float32) 

# Preprocess the data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Define the Bayesian logistic regression model
def logistic_regression(X, w, b):
    logits = tf.matmul(X, w) + b
    return tfd.Bernoulli(logits=logits)

# Create the variables w and b outside the function
w = tf.Variable(tf.zeros([X_train.shape[1], 1]), name='w')
b = tf.Variable(0., name='b')

# Define the joint log probability
def joint_log_prob(X, y, w, b):
    prior_w = tfd.Normal(loc=tf.zeros_like(w), scale=tf.ones_like(w))
    prior_b = tfd.Normal(loc=tf.zeros_like(b), scale=tf.ones_like(b))
    likelihood = logistic_regression(X, w, b)  # Pass w and b as arguments
    return tf.reduce_sum(prior_w.log_prob(w)) + prior_b.log_prob(b) + tf.reduce_sum(likelihood.log_prob(y))

# Define the loss function
def negative_log_joint(X, y, w, b):
    return -joint_log_prob(X, y, w, b)

# Optimize the model
optimizer = tf.optimizers.Adam(learning_rate=0.01)
trainable_variables = [w, b]
for i in range(1000):
    with tf.GradientTape() as tape:
        loss = negative_log_joint(X_train, y_train[:, np.newaxis], trainable_variables[0], trainable_variables[1])
    grads = tape.gradient(loss, trainable_variables)
    optimizer.apply_gradients(zip(grads, trainable_variables))

# Make predictions on the test set
test_logits = tf.matmul(X_test, trainable_variables[0]) + trainable_variables[1]
y_test_preds = tf.round(tf.sigmoid(test_logits)).numpy().flatten()

# Calculate accuracy
accuracy = accuracy_score(y_test, y_test_preds)

# Print the accuracy
print("Accuracy with Bayesian Logistic Regression: {:.2f}%".format(accuracy * 100))



Accuracy with Bayesian Logistic Regression: 98.25%
